In [181]:
import numpy as np
import pandas as pd
from faker import Faker

In [182]:
paper = pd.read_csv('../data/publication/papers.csv', sep=';')
author = pd.read_csv('../data/publication/authors.csv', sep=',')
paper_author = pd.read_csv('../data/publication/paper_written_by_author.csv', sep=';', dtype={'paper_id':str})

In [183]:
author.organization = author.organization.replace('[^0-9a-zA-Z]+',' ', regex=True)
result_df = pd.merge(pd.merge(paper_author,author, on='author_id'),paper, on='paper_id')
result_df = result_df.rename(columns={'name': 'author'})

In [184]:
result_df = result_df.groupby('paper_id').agg({'author': lambda x: '|'.join(map(str, x)), 'title':'first', 'year':'first'}).reset_index()

In [185]:
keyword = pd.read_csv('../data/publication/keywords.csv')
paper_keyword = pd.read_csv('../data/publication/paper_has_keywords.csv', dtype={'paper_id':str})

In [186]:
result_df = pd.merge(pd.merge(paper_keyword,keyword,on='keyword_id').groupby('paper_id').agg({'keyword': lambda x: '|'.join(map(str, x))}).reset_index(),result_df, on='paper_id')

In [187]:
conference = pd.read_csv('../data/publication/conference.csv', sep=';')
conference = conference.rename(columns={'name':'venue'})
paper_conf = pd.read_csv('../data/publication/paper_published_in_conference.csv', sep=';', dtype={'paper_id':str})

In [188]:
temp1 = pd.merge(pd.merge(paper_conf, conference, on='conference_id'),result_df,on='paper_id')
temp1 = temp1.drop('conference_id', axis=1)

In [189]:
journal = pd.read_csv('../data/publication/journal.csv', sep=';')
journal = journal.rename(columns={'name':'venue'})
paper_jour = pd.read_csv('../data/publication/paper_published_in_journal.csv', sep=';', dtype={'paper_id':str})

In [190]:
temp2 = pd.merge(pd.merge(paper_jour, journal, on='journal_id'),result_df,on='paper_id')
temp2 = temp2.drop('journal_id', axis=1)

In [191]:
temp1['venue_type'] = 'Conference'
temp2['venue_type'] = 'Journal'
temp2['conf_type'] = None

In [192]:
values = ['Symposium','Regular Conference','Workshop','Expert Group']
temp1['conf_type'] = np.random.choice(values, size=len(temp1))

In [193]:
result_df = pd.concat([temp1.iloc[:1000],temp2.iloc[:1000]])

In [194]:
fake = Faker()
decision = ['ACCEPTED','REJECTED']
pub_type = ['Conference Proceeding', 'Volumne']
p_type = ['Full Paper', 'Short Paper', 'Demo Paper', 'Poster']
result_df['review_text'] = [fake.unique.text() for _ in range(len(result_df))]
result_df['abstract'] = [fake.unique.text() for _ in range(len(result_df))]
result_df['venue_city'] = [fake.city() for _ in range(len(result_df))]
result_df['head'] = [fake.unique.name() for _ in range(len(result_df))]
result_df['reviewer1'] = [fake.unique.name() for _ in range(len(result_df))]
result_df['reviewer2'] = [fake.unique.name() for _ in range(len(result_df))]
result_df['decision'] = np.random.choice(decision, size=len(result_df))
result_df['pub_type'] = np.random.choice(pub_type, size=len(result_df))
result_df['paper_type'] = np.random.choice(p_type, size=len(result_df))

In [195]:
result_df.year = result_df.year.fillna(2012)
result_df.year = result_df.year.astype(int)
result_df.loc[result_df.paper_type == "Poster", "venue_type"] = "Conference"

In [196]:
result_df.to_csv('../data/rdf_instances_new.csv', index=False)

In [197]:
result_df.columns

Index(['paper_id', 'venue', 'publication', 'keyword', 'author', 'title',
       'year', 'venue_type', 'conf_type', 'review_text', 'abstract',
       'venue_city', 'head', 'reviewer1', 'reviewer2', 'decision', 'pub_type',
       'paper_type'],
      dtype='object')

In [203]:
print(any(result_df.paper_id.isnull()))
print(any(result_df.venue.isnull()))
print(any(result_df.publication.isnull()))
print(any(result_df.keyword.isnull()))
print(any(result_df.author.isnull()))
print(any(result_df.title.isnull()))
print(any(result_df.year.isnull()))
print(any(result_df.venue_type.isnull()))
print(any(result_df.conf_type.isnull()))
print(any(result_df.review_text.isnull()))
print(any(result_df.abstract.isnull()))
print(any(result_df.venue_city.isnull()))
print(any(result_df['head'].isnull()))
print(any(result_df.reviewer1.isnull()))
print(any(result_df.reviewer2.isnull()))
print(any(result_df.decision.isnull()))
print(any(result_df.pub_type.isnull()))

False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False


<bound method IndexOpsMixin.value_counts of 0               Symposium
1            Expert Group
2      Regular Conference
3               Symposium
4                Workshop
              ...        
995                  None
996                  None
997                  None
998                  None
999                  None
Name: conf_type, Length: 2000, dtype: object>